In [86]:
import geopandas
import pathlib
import shapely

In [87]:
base_path = pathlib.Path(r"C:\Local\data\river_bathemetry\sections\Westport")

# Split out different sections depending on the test dataset
### Small
Includes sections Section7, Section8, Section9, Section10

In [88]:
sections = geopandas.read_file(base_path / "river_outlines" / "small" / "measured_sections.geojson")
riverbanks = geopandas.read_file(base_path / "river_outlines" / "small" / "river_banks.geojson")

# Explore Interpolation between sections
1. Estimate the centreline from the polygon
2. Create profiles from the centerline
3. Estimate the Thalweg from the measued sections
4. Interpolate along points sampled along the lines

In [93]:
riverbanks

,Name,geometry
0,Right,"LINESTRING (1483413.197 5375259.983, 1483663.4..."
1,Left,"LINESTRING (1483187.524 5375194.332, 1483229.3..."
